In [1]:
####This part is a ready-to-use block, which makes it possible to import ipynb in Jupyter notebook.
##Please ignore this block!
##Please ignore this block!
##Please ignore this block!
##Please ignore this block!
import io, os, sys, types
from IPython import get_ipython
from nbformat import current
from IPython.core.interactiveshell import InteractiveShell


def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path


class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = current.read(f, 'json')


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.worksheets[0].cells:
                if cell.cell_type == 'code' and cell.language == 'python':
                # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.input)
                # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)


In [2]:
#Import unittest module and the module need to be tested.
import unittest
import macdyc

importing Jupyter notebook from macdyc.ipynb


In [3]:
##From now we begin the unit testing of some blocking with unittest module.
class macdTestCase(unittest.TestCase):
    '''Tests for macdyc module.'''
    def test_MA(self):
        '''Test for the Moving Average (MA) fuction.'''
        self.assertEqual(list(macdyc.MA([1,2,3],0)),[1,2,3])
        self.assertEqual(list(macdyc.MA([2,2,2],2)),[2,2,2])
    def test_crossover(self):
        '''Test for the crossover fuction, which is used to compute diff-sig.'''
        self.assertEqual(list(macdyc.crossover([1,2,3],0,0,0)),[0,0,0])
        self.assertEqual(list(macdyc.crossover([1,1,1],2,2,2)),[0,0,0])
    def test_ratio(self):
        '''Test for the ratio fuction, which is used to indicate the change of an array!'''
        self.assertEqual(macdyc.ratio([2,2,3,4]),[1,1,1.5,2])
        self.assertEqual(macdyc.ratio([5,6,7,4]),[1,1.2,1.4,0.8])
    def test_actions(self):
        '''Test for the actions fuction, which can return the Buy-Sell sequence!'''
        self.assertEqual(macdyc.actions([0,1,2,3,2,1,-1]),['nochange','buy','nochange','nochange',\
                                                         'nochange','nochange','sell',])
    def test_valuedetail(self):
        '''Test valuedetail, which return revenue details with the given Buy-Sell sequence!'''
        action=['nochange','buy','nochange','nochange','sell']
        data=[1,1.5,3,6,1.5]
        initialvalue=1000000
        commissionrate=0
        self.assertEqual(macdyc.valuedetail(action,data,initialvalue,commissionrate)[0],[1000000,1000000,\
                                                                                          2000000,4000000,1000000])
unittest.main(argv=['first-arg-is-ignored'], exit=False)

macdyc.ipynb:4: FutureWarning: pd.ewm_mean is deprecated for ndarrays and will be removed in a future version
  "cell_type": "code",
.....
----------------------------------------------------------------------
Ran 5 tests in 0.104s

OK
